In [ ]:
!pip install num2words
!pip install nltk
!pip install pip beautifulsoup4
!pip install pip WordCloud
!pip install pip Counter
!pip install pip Word2Vec
!pip install scikit-learn
!pip install gensim

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=38d230db7ee770091ce516860fc9457a5f8a74ccd7bdebf29f40079bf4a7f776
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Preparing metadata (setup.py) ... done
  Created wheel for Counter: filename=Counter-1.0.0-py3-none-any.whl size=5394 sha256=679089ff2bb63805ceb390e2bec0913ba2a0176b17203436f8e318c3226de329
  Stored in directory: /root/.cache/pip/wheels/e3/02/6d/d5c0838427a060718c6060ae4d24da95a0e0df0d7a3dab8040
Successfully built Counter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import pandas as pd
import unicodedata
from num2words import num2words
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from collections import Counter
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk import ngrams

from collections import Counter
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

nltk_sw_list = stopwords.words('english')
nltk_lematizer_list = WordNetLemmatizer()

In [ ]:
# Funcion para Preprocesado


def eliminar_stop_words(texto):
    tokens = word_tokenize(texto)
    tokens = [word for word in tokens if word not in nltk_sw_list]
    return tokens

def eliminar_signos_puntuacion(texto):
    text = re.sub(r'[^a-z\s]', '', texto)
    return text

def eliminar_casos_especiales(texto):
    texto= texto.replace('<br />',' ')
    return texto

def elimimnar_etiquetas_html(texto):
   text = BeautifulSoup(texto, "html.parser").get_text()
   texto = re.sub(r'<.*?>', '', texto)
   return texto

def revision_generica(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = text.lower()
    return text


def revision_lematizer(tokens):
    tokens = [nltk_lematizer_list.lemmatize(word) for word in tokens]
    return tokens


def texto_preproceado(text):
    clean_text = list()

    text = revision_generica(text)
    text = elimimnar_etiquetas_html( text)
    text = eliminar_signos_puntuacion(text)
    text = eliminar_casos_especiales(text)
    tokens = eliminar_stop_words(text)
    tokens = revision_lematizer(tokens)

    clean_text = ' '.join(tokens)

    return clean_text